In [3]:
from tkinter import *
from tkinter import ttk
import sqlite3
from pprint import pprint 
from datetime import datetime

#A ocr/tesseract-hoz
import cv2
import pytesseract
import numpy as np
from PIL import ImageGrab
import time


conn= None
curs= None





def db_create(): # adatbázis létrehozása
    global conn, curs
    conn=sqlite3.connect("kocsik.db")
    curs= conn.cursor()
    
    curs.execute("CREATE TABLE IF NOT EXISTS kocsik (Rszam TEXT, ido INT)") #kocsik adattábla létrehozása
    
def db_insert(rszam): #adatbeszúrás az adatbázisba.
    now = datetime.now()
    most = now.strftime("%H:%M:%S") #pillanatnyi időt kéri le
    curs.execute("SELECT * FROM kocsik WHERE Rszam=?",(rszam.get(),))
    datas= curs.fetchall()
    if(len(datas)==0):    
        curs.execute("INSERT INTO kocsik VALUES (?,?)",(rszam.get(),most)) #alap sql parancs a beszúrásra.
        conn.commit() #adatbázis "mentése"
        rszam.delete(0,'end')#hozzáadás után a beírt rendszám törlődjön.
        db_query(table) # query-t meghívjuk újra, hogy a táblázatban már a friss adatok legyenek.
    else: #Ha már létezik ilyen rendszám, írja ki hogy: Már van ilyen rendszám. Ezt nem adja hozzá.
        table.delete(*table.get_children())
        table.insert("","end",values=("","Már van ilyen rendszámú elem.",""))
    
def db_query(table): #adatok lekérése
    curs.execute("SELECT *FROM kocsik")#alap sql parancs
    datas= curs.fetchall()
    table.delete(*table.get_children()) #Erre azért van szükség, hogy a lekérés előtt a tábla törlődjön.
    rowid=1
    for data in datas:
        table.insert("","end",values=(rowid,data[0],data[1]))#táblázatba írás folyamata
        rowid=rowid+1


def db_close(): #adatbázis lezárása
    if conn and curs:
        conn.close() # adatbázis lezárása
def db_delete(rendszam): #Adatok törlése az adatbázisból.
    curs.execute("DELETE FROM kocsik WHERE Rszam=?",(rendszam.get(),))
    db_query(table) #Táblázatban a friss adatok jelenjenek meg...
    rendszam.delete(0,'end') #gomb lenyomása után a szövegdoboz törlődjön.
    conn.commit() #Adatbázis mentése.
def db_keres(rendszam): #Keresés funkció
    table.delete(*table.get_children())
    curs.execute("SELECT * FROM kocsik WHERE Rszam=?",(rendszam.get(),)) #Alap sql parancs
    datas=curs.fetchall()
    if(len(datas)==0): #Ha keresésnél nem talál ilyen rendszámot, akkor írja ki hogy: Nincs ilyen elem
        table.insert("","end",values=("","Nincs ilyen elem",""))
    rowid=1
    for data in datas:
        table.insert("","end",values=(rowid,data[0],data[1]))
def db_keprendszam(text):
    now = datetime.now()
    most = now.strftime("%H:%M:%S")
    curs.execute("SELECT * FROM kocsik WHERE Rszam=?",(text,))
    datas= curs.fetchall()
    if(len(datas)==0):   
        curs.execute("INSERT INTO kocsik VALUES(?,?)",(text,most))
        conn.commit()
    

db_create()        
#RENDSZÁM felismerés/olvasása
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe'
 
img = cv2.imread('kep.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
text=pytesseract.image_to_string(img)
print(text)
 
#cv2.imshow('img', img)
db_keprendszam(text) #Itt rakja be a képről a rendszámot az adatbázisba.


#Ablak létrehozása/beállításai:
win = Tk()
win.title("Személygépjármű adatbázisa")
win.geometry('1200x560') # ablak mérete
win.resizable(False,False) #ne lehessen változtatni a méretét, persze ezt ki is lehet kapcsolni


#Label = szöveg, Entry=szövegdoboz, Button=gomb, grid()-ben van megadva hol helyezkedjen el (táblázatszerűen van felosztva)
Label(win, text="Garázs adatbázis", font="Helvetica 12 bold").grid(row=0 ,column=2)
#Műveletek:
#Keresés:
Label(win, text="Keresés rendszám alapján: ", font="Helvetica 10 bold").grid(row=1 ,column=0)
Label(win, text="Rendszám: ", font="Helvetica 10 bold").grid(row=2 ,column=0)
rendszam=Entry(win, font="Helvetica 10 bold")
rendszam.grid(row=2,column=1, padx=5, pady=5)
Button(win, text="Keresés",command=lambda: db_keres(rendszam), font="Helvetica 10 bold").grid(row=2,column=2,padx=5,pady=5)

#Hozzáadás:
Label(win, text="Rendszám hozzáadása: ", font="Helvetica 10 bold").grid(row=3 ,column=0)
Label(win, text="Rendszám: ", font="Helvetica 10 bold").grid(row=4 ,column=0)
rszam=Entry(win, font="Helvetica 10 bold")
rszam.grid(row=4,column=1, padx=5, pady=5)
Button(win, text="Hozzáad",command=lambda: db_insert(rszam), font="Helvetica 10 bold").grid(row=4,column=2,padx=5,pady=5)

#Törlés
Label(win, text="Rendszám törlése: ", font="Helvetica 10 bold").grid(row=5 ,column=0)
Label(win, text="Rendszám: ", font="Helvetica 10 bold").grid(row=6 ,column=0)
rszam2=Entry(win, font="Helvetica 10 bold")
rszam2.grid(row=6,column=1, padx=5, pady=5)
Button(win, text="Törlés",command=lambda: db_delete(rszam2), font="Helvetica 10 bold").grid(row=6,column=2,padx=5,pady=5)

#Összes listázása:
Label(win, text="Összes parkolt autó listázása: ", font="Helvetica 12 bold").grid(row=7 ,column=1)
Label(win, text="Többváltozós analízis mérnöki alkalmazásai tantárgy projekt céljából készült.  ", font="Helvetica 10 bold").grid(row=8 ,column=0,columnspan=3,padx=5)
#Táblázat, amiben megjelennek az adatok.
cols= ("ID","Rendszám","parkolás ideje")
table=ttk.Treeview(win, columns=cols,show="headings")
for col in cols:
    table.heading(col, text=col)
table.grid(row=2,column=3,rowspan=5,padx=5,pady=5)
Button(win, text="Autók listázása",command=lambda: db_query(table), font="Helvetica 10 bold").grid(row=7,column=2,padx=5,pady=5)

win.mainloop()
db_close()

ABC-123


[]
